- Joel Konitzer
- 3/20/2022
- Week 2 / Probability and Proportion
- Data Analytics

I. Introduction: The purpose of this analysis is to generate analytics from histoircal flight data in order to deteremine the prerfered airport to depart from. The data has been cleaned and preped for this analysis, and each row represents unique flight information. By applying python functions to calcuatie the proportion, probability, mean, and standard deviation of each airport, based on hsistorical flight data, we should be able to determine our departing flight.

*II. Methods & Code **III. Results/Output ***IV. Analysis of Results

In [355]:
import pandas as pd
import numpy as np

In [378]:
df = pd.read_csv('Data/flights_clean.csv')

In [379]:
#Preview dataframe
df.head()

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0


In [380]:
#Create two value pairs from df to identify the datatypes and null values
null_ = df.isna().any()
dtypes = df.dtypes
info = pd.concat([null_,dtypes], axis=1,keys=['Null','type'])
print(info)
print('======================================================')
#View dataframe shape
print("Dataframe shape:", df.shape)

                      Null     type
year                 False    int64
month                False    int64
day                  False    int64
day_of_week          False    int64
airline              False   object
flight_number        False    int64
origin_airport       False   object
destination_airport  False   object
scheduled_departure  False    int64
departure_time       False  float64
departure_delay      False  float64
scheduled_time        True  float64
elapsed_time          True  float64
scheduled_arrival    False    int64
arrival_time          True  float64
arrival_delay        False  float64
diverted             False    int64
cancelled            False    int64
Dataframe shape: (5245484, 18)


In [381]:
#Create new dataframe that filters for delayed flights
delayed = df[df.arrival_delay > 0].groupby('origin_airport')['arrival_delay'].count().reset_index()
delayed['Total_Delayed'] = delayed.arrival_delay.sum()

1. Determine the originaing airport with the largest proportion of flights arriving late to their destination. Do the same for the airport with the smallest proportion.

In [382]:
#Calculate the airport with the highest proportion of delayed flights
delayed['Delayed_Prop'] = delayed.arrival_delay / delayed.Total_Delayed
delayed.sort_values('Delayed_Prop', ascending=False).head(1)

,origin_airport,arrival_delay,Total_Delayed,Delayed_Prop
20,ATL,115034,1947990,0.059053


* The originating airport with the highest proportion of delayed flights is ATL with a total of 5% of their flights being delayed.

1b. do the same for the airport with the smallest proportion.

In [383]:
#Calculate the airport with the lowest proportion of delayed flights
delayed.sort_values('Delayed_Prop', ascending=True).head(1)

,origin_airport,arrival_delay,Total_Delayed,Delayed_Prop
70,CNY,8,1947990,0.000004


* The originating airport with the highest proportion of delayed flights is CNY with a total of 0.00004% of their flights being delayed.

2. What is the probablity a flight leaving from a given airport will arrive at its destination late?
- Hint: Calculate the probablity of late arrival at destination for each originating airport.

In [384]:
delayed = df[df.arrival_delay > 0].groupby('origin_airport')['arrival_delay'].count()
groupedbyFlight = df.groupby('origin_airport')['arrival_delay'].count()

In [385]:
#Calculation the probability of delayed arrival flights 
df_prob = delayed / groupedbyFlight
df_prob = df_prob.reset_index().sort_values('arrival_delay', ascending=False)
cols = ['Origin_Airport','Prob_ArriveLate']
df_prob.columns = cols

In [386]:
#Calculate and round the total average probability of delayed arrival flights
print('==============================================================================================================')
print('Total average probability of a flight leaving from a given airport will arrive at its destination late: ', round(df_prob.Prob_ArriveLate.mean(), 3))
print('==============================================================================================================')
df_prob

Total average probability of a flight leaving from a given airport will arrive at its destination late:  0.346


,Origin_Airport,Prob_ArriveLate
133,GST,0.671053
9,ADK,0.640449
246,PPG,0.626168
154,ILG,0.557895
13,AKN,0.507937
...,...,...
88,DIK,0.207383
189,LWS,0.164966
46,BTM,0.152778
314,VEL,0.115000


The total average probability of a flight arriving at its destination late is 34.6%. The originating airport with the highest probability of a flight arriving late to its destination is GST followed by ADK, both of these are in Alaska.

3. What is the mean and std of late arrival times for both of these airports
- Based on the mean and std information ONLY, which airport seems like a better choice?

In [387]:
#Group the data by origin_airport and aggregate the data to calculate mean and standard standard deviation
grouped = df[df.arrival_delay > 0].groupby('origin_airport')
df_grouped = grouped['arrival_time'].agg([np.mean, np.std]).reset_index()
df_prob[['Mean_ArriveLate', 'STD_ArriveLate']] = df_grouped[['mean', 'std']]

Mean of late arrival times, descending order

In [388]:
#Calculate the standard deviation
df_prob[['Origin_Airport', 'Mean_ArriveLate', 'Prob_ArriveLate']].sort_values('Mean_ArriveLate',ascending=False)

,Origin_Airport,Mean_ArriveLate,Prob_ArriveLate
9,ADK,2141.491228,0.640449
313,UST,1922.950820,0.423611
133,GST,1851.196078,0.671053
13,AKN,1838.062500,0.507937
219,MVY,1812.385714,0.341463
...,...,...,...
168,JMS,879.023077,0.346614
234,PBG,857.738462,0.465950
42,BQN,831.312775,0.340841
248,PSE,819.868132,0.246612


STD of late arrival times, descending order

In [389]:
#Calculate the standard deviation
df_prob[['Origin_Airport', 'STD_ArriveLate', 'Prob_ArriveLate']].sort_values('STD_ArriveLate',ascending=False)

,Origin_Airport,STD_ArriveLate,Prob_ArriveLate
154,ILG,777.867633,0.557895
166,JFK,697.138470,0.364821
203,MIA,667.308383,0.405146
195,MDW,666.939960,0.404196
114,FLL,665.127546,0.340711
...,...,...,...
70,CNY,134.480947,0.039024
231,OTH,125.335238,0.433962
133,GST,63.396536,0.671053
89,DLG,54.869431,0.220779


It appears that the standard deviation would be a better metric to go by considering the highest STD value has a lower probability of arriving late than the highest mean does.

4. Define a question that would utilize Bernoulli's Equation and preform a calculation to support your question.

In [390]:
from scipy import stats

Q. I need to fly out of Chicago 5 times during the month of May, what Chicagoland airport should I choose so that reduce the chances of having a delayed flight, and if I do, I will at least have a shorter delay?

In [391]:
#List of Chicagoland airports (to include Milwaukee)
airports_q = ['ORD' ,'MDW', 'MKE']

In [392]:
#Calculate the probability mass function for the airports of interest
df_prob['BinaryDist_Prob'] = df_prob.Prob_ArriveLate.apply(lambda x: stats.binom.pmf(1, n=5, p=x))

In [393]:
#Filter to view only airports of interest
df_prob[df_prob.Origin_Airport.isin(airports_q)].copy()

,Origin_Airport,Prob_ArriveLate,Mean_ArriveLate,STD_ArriveLate,BinaryDist_Prob
228,ORD,0.408937,1564.041881,583.924122,0.249552
195,MDW,0.404196,1515.746364,666.939960,0.254668
204,MKE,0.327202,1530.420926,506.791787,0.335216


- Using the probability mass function to deteremine the our late flight arrival probability, we see that ORD will actually give us a lower probability than both MKE and MDW.
- The probability of having 1 flight delayed out of the 5 I plan to take is 24% if I decide to choose ORD

5. Provide a summary of all the values that you calculated for 3 airports

In [394]:
#Filter to view only airports of interest
airports = ['ATL', 'CNY', 'GST']
df_prob[df_prob.Origin_Airport.isin(airports)].copy()

,Origin_Airport,Prob_ArriveLate,Mean_ArriveLate,STD_ArriveLate,BinaryDist_Prob
133,GST,0.671053,1851.196078,63.396536,0.039285
20,ATL,0.334130,1567.160106,583.868142,0.328430
70,CNY,0.039024,1337.375000,134.480947,0.166401


- Compare the three to each other.
- Which airport would you prefer to fly out of based on your results.

The three airports chosen were the airports with extreme  mean, std, and probability values. Each airport showed to have similar total mean arrival times, but very different standard deviations. Based on these, I would have to choose CNY since it has a much lower standard deviation of late arrival times as well as a much lower probability of being late. 

V. Conclusion

This analysis showed that it is less helpful to use late arrival probability to determine a departing airport than to use mean late arrival time and standard deviation. Additionally, using the binomial distribution functions provided by pandas, we can anticipate our expectations around arrival times given a set of conditions. Important items to discuss for this analysis is that late arrivals are considered even if a flight is 1 minute late, this is responsible for the high probability of late arrivals but low mean arrival time for airports like ATL or ORD. The arrivals with the highest mean arrival time were geographically isolated airports, with lower standard deivation. 

VI. Sources

- Pandas.dataframe.round¶. pandas.DataFrame.round - pandas 1.4.1 documentation. (n.d.). Retrieved March 20, 2022, from https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.round.html 
- Python Pandas - GroupBy. Python pandas - groupby. (n.d.). Retrieved March 20, 2022, from https://www.tutorialspoint.com/python_pandas/python_pandas_groupby.htm 